The goal of this notebook is to take the ad hoc implementation in `MIPExample1.ipynb` and vectorize the operations such that any size Tyche problem can be converted to a MIP and solved.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

In [3]:
import cProfile
import timeit

In [4]:
from mip import Model, MINIMIZE, MAXIMIZE, BINARY, xsum
from itertools import product

In [5]:
designs = ty.Designs("data")
investments = ty.Investments("data")
designs.compile()
tranche_results = investments.evaluate_tranches(designs, sample_count=250)
results = investments.tranches.join(tranche_results.summary)
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Get the wide-format interpolated elicitation data from the Tyche Evaluator and reset the multi-level index.

In [6]:
wide = evaluator.evaluate_corners_wide().reset_index()

In [7]:
wide.head(9)

Index,CIGS,CdTe,GaAs,InGaP,Perovskite,Polysilicon,Power Electronics,Soft Costs,Capital,Efficiency,GHG,Hazardous,LCOE,Lifetime,Strategic,Yield
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.630415,2.059875,-0.003592,0.978331,-0.148178,187.908287,0.063718,9999.360891
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,-1.537077,2.060912,-0.003592,0.977261,-0.145257,187.908240,0.063718,9999.359819
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,-1.372341,2.060339,-0.003592,0.977821,-0.140103,187.908256,0.063718,9999.363456
3,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,0.0,-1.545457,2.061322,-0.003592,0.965075,-0.141841,187.908250,0.063718,10063.083397
4,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1000000.0,-1.452120,2.062359,-0.003592,0.964004,-0.138921,187.908203,0.063718,10063.082324
5,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,5000000.0,-1.287384,2.061787,-0.003592,0.964564,-0.133766,187.908219,0.063718,10063.085962
6,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,0.0,-1.468591,2.061035,-0.003592,0.957985,-0.137301,187.908165,0.063718,10106.656795
7,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,1000000.0,-1.375253,2.062072,-0.003592,0.956914,-0.134381,187.908118,0.063718,10106.655723
8,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,5000000.0,-1.210517,2.061500,-0.003592,0.957474,-0.129226,187.908133,0.063718,10106.659360


**Input to MIP constructor needed**: List of investment categories

**Input to MIP constructor needed**: List of metrics to optimize and/or constrain

**Data check**: Confirm that all elements of both input lists match columns in the `evaluate_corners_wide()` data frame.

In [8]:
categories = ['Polysilicon', 'Power Electronics', 'Soft Costs']

metrics = ['Capital']

Pull out the investment values and metric values from the data set.

In [9]:
# Investment levels
inv_levels = wide.loc[:26,categories].values.tolist()

# Elicited metric values
m = wide.loc[:26,metrics].values.tolist()

# Number of endpoints
I = len(inv_levels)

Instantiate the MIP optimization problem.

In [25]:
example = Model(sense=MAXIMIZE)

Create binary (integer) variables $y_{in_i}$.

In [26]:
bin_vars = []

for i in range(I-1):
    _name = 'y' + '_' + str(i)
    bin_vars += [example.add_var(name=_name, var_type=BINARY)]
    del _name

Create continuous $\lambda$ variables with lower bound 0.0 and upper bound 1.0

In [27]:
lmbd_vars = []

for i in range(I):
    _name = 'lmbd' + '_' + str(i)
    lmbd_vars += [example.add_var(name=_name, lb=0.0, ub=1.0)]
    del _name

Define upper bound for total budget constraint and per-category upper bounds.

In [28]:
B = 3000000.0
max_amount = [1000000.0, 2000000.0, 1500000.0]

Create total budget constraint as a function of the $\lambda$ variables and the elicited investment levels from the data set.

In [29]:
example += xsum(lmbd_vars[i] * inv_levels[i][j] 
                for i in range(I)
                for j in range(len(inv_levels[i]))) <= B, 'Total Budget'

Create constraints for budget within each investment category.

In [30]:
for j in range(len(categories)):
    example += xsum(lmbd_vars[i] * [el[j] for el in inv_levels][i] 
                    for i in range(I)) <= max_amount[j], 'Budget for ' + categories[j]

Create three sets of constraints:
* Convexity constraints on $\lambda$ variables: the $\lambda$s must sum to 1.
* Constrain binary $y$ variables within each category such that the decision variables lie within exactly one interval (also a sum-to-1 constraint).
* Interval constraints on $y$ variables and $\lambda$ variables. There will be as many interval constraints per investment category as there are linear intervals.

In [31]:
example += sum(lmbd_vars) == 1, 'Lambda Sum'
example += sum(bin_vars) == 1, 'Binary Sum'

In [32]:
for i in range(I-1):
    example += bin_vars[i] <= lmbd_vars[i] + lmbd_vars[i+1], 'Interval Constraint' + str(i)

**In progress/Not implemented from this point onward**

Create objective function: Capital (metric) as a function of $\lambda$s and $y$s.

Create a data frame of $\lambda$ variables by mapping every element in `v` to its corresponding $\lambda$, using column name (investment category) and investment level.

In [33]:
example.objective = xsum(m[i][0] * lmbd_vars[i] for i in range(I))

Save model to a `lp` file for manual examination.

In [34]:
example.write('example.lp')

Optimize.

In [35]:
example.optimize()

<OptimizationStatus.OPTIMAL: 0>

Print optimal objective function value, with a negative applied to reverse the -1.0 in the objective function definition (minimizing a negative -> maximize a positive).

In [36]:
example.objective_value

-1.4315276524348488

Get the optimal $\lambda$ and $y$ values.

In [37]:
for v in example.vars:
    print('{} : {}'.format(v.name, v.x))

y_0 : 0.0
y_1 : 0.0
y_2 : 0.0
y_3 : 0.0
y_4 : 1.0
y_5 : 0.0
y_6 : 0.0
y_7 : 0.0
y_8 : 0.0
y_9 : 0.0
y_10 : 0.0
y_11 : 0.0
y_12 : 0.0
y_13 : 0.0
y_14 : 0.0
y_15 : 0.0
y_16 : 0.0
y_17 : 0.0
y_18 : 0.0
y_19 : 0.0
y_20 : 0.0
y_21 : 0.0
y_22 : 0.0
y_23 : 0.0
y_24 : 0.0
y_25 : 0.0
lmbd_0 : 0.0
lmbd_1 : 0.0
lmbd_2 : 0.0
lmbd_3 : 0.0
lmbd_4 : 0.875
lmbd_5 : 0.125
lmbd_6 : 0.0
lmbd_7 : 0.0
lmbd_8 : 0.0
lmbd_9 : 0.0
lmbd_10 : 0.0
lmbd_11 : 0.0
lmbd_12 : 0.0
lmbd_13 : 0.0
lmbd_14 : 0.0
lmbd_15 : 0.0
lmbd_16 : 0.0
lmbd_17 : 0.0
lmbd_18 : 0.0
lmbd_19 : 0.0
lmbd_20 : 0.0
lmbd_21 : 0.0
lmbd_22 : 0.0
lmbd_23 : 0.0
lmbd_24 : 0.0
lmbd_25 : 0.0
lmbd_26 : 6.577832330824488e-17


Verify that budget constraint is met.

Calculate the optimal investment levels.